# Exercício - Mini Projeto de Análise de Dados

Vamos fazer um exercício completo de pandas para um miniprojeto de análise de dados.

Esse exercício vai obrigar a gente a usar boa parte dos conhecimento de pandas e até de outros módulos que já aprendemos ao longo do curso.

### O que temos?

Temos os dados de 2019 de uma empresa de prestação de serviços. 

- CadastroFuncionarios
- CadastroClientes
- BaseServiçosPrestados

Obs1: Para ler arquivos csv, temos o read_csv<br>
Obs2: Para ler arquivos xlsx (arquivos em excel normais, que não são padrão csv), temos o read_excel

### O que queremos saber/fazer?

1. Valor Total da Folha Salarial -> Qual foi o gasto total com salários de funcionários pela empresa? <br>
    Sugestão: calcule o salário total de cada funcionário, salário + benefícios + impostos, depois some todos os salários
    
    
2. Qual foi o faturamento da empresa?<br>
    Sugestão: calcule o faturamento total de cada serviço e depois some o faturamento de todos
    
    
3. Qual o % de funcionários que já fechou algum contrato?<br>
    Sugestão: na base de serviços temos o funcionário que fechou cada serviço. Mas nem todos os funcionários que a empresa tem já fecharam algum serviço.<br>
    . Na base de funcionários temos uma lista com todos os funcionários<br>
    . Queremos calcular Qtde_Funcionarios_Fecharam_Serviço / Qtde_Funcionários_Totais<br>
    . Para calcular a qtde de funcionários que fecharam algum serviço, use a base de serviços e conte quantos funcionários tem ali. Mas lembre-se, cada funcionário só pode ser contado uma única vez.<br><br>
    Dica: se você aplicar o método .unique() em uma variável que é apenas 1 coluna de um dataframe, ele vai excluir todos os valores duplicados daquela coluna.<br>
    Ex: unicos_colunaA = dataframe['colunaA'].unique() te dá como resposta uma lista com todos os itens da colunaA aparecendo uma única vez. Todos os valores repetidos da colunaA são excluidos da variável unicos_colunaA 
    
    
4. Calcule o total de contratos que cada área da empresa já fechou


5. Calcule o total de funcionários por área


6. Qual o ticket médio mensal (faturamento médio mensal) dos contratos?<br>
    Dica: .mean() calcula a média -> exemplo: media_colunaA = dataframe['colunaA'].mean()

Obs: Lembrando as opções mais usuais de encoding:<br>
encoding='latin1', encoding='ISO-8859-1', encoding='utf-8' ou então encoding='cp1252'

Observação Importante: Se o seu código der um erro na hora de importar os arquivos:<br>
- CadastroClientes.csv
- CadastroFuncionarios.csv

Use separador ";" (ponto e vírgula) para resolver

### Importação de Módulos e Arquivos

In [6]:
import pandas as pd

funcionarios_df = pd.read_csv(r'CadastroFuncionarios.csv', sep=';', decimal=',')
clientes_df = pd.read_csv(r'CadastroClientes.csv', sep=';')
servicosPrestados_df = pd.read_excel(r'BaseServiçosPrestados.xlsx')


### 1 - Folha Salarial

In [63]:
# Minha resolução para o primeiro problema

# limpar base de dados, retirando colunas que não serão usadas

#funcionarios_df = funcionarios_df.drop(['Estado Civil', 'Cargo'], axis=1)

#funcionarios_df.info()

vlrTotalFolha = funcionarios_df.groupby('Nome Completo').sum()

vlrTotalFolha = vlrTotalFolha['Salario Base'].sum() + vlrTotalFolha['Impostos'].sum() + vlrTotalFolha['Beneficios'].sum() + vlrTotalFolha['VT'].sum() + vlrTotalFolha['VR'].sum()

vlrTotalFolha = 'R${:_.2f}'.format(vlrTotalFolha)
vlrTotalFolha = vlrTotalFolha.replace('.', ',').replace('_', '.')
print("O valor total da folha foi de: {}".format(vlrTotalFolha))

O valor total da folha foi de: R$2.717.493,22


In [64]:
#Resolução do professor da Aula
funcionarios_df['Salario Total'] = funcionarios_df['Salario Base'] + funcionarios_df['Impostos'] + funcionarios_df['Beneficios'] + funcionarios_df['VT'] + funcionarios_df['VR']
print('Total da Folha Salarial Mensal é de R${:,}'.format(funcionarios_df['Salario Total'].sum()))

Total da Folha Salarial Mensal é de R$2,717,493.22


### 2 - Faturamento da Empresa

In [24]:
#Minha resolução para o segundo problema
tabaux_faturamento_df = servicosPrestados_df[['ID Cliente', 'Tempo Total de Contrato (Meses)']].merge(clientes_df[['ID Cliente','Valor Contrato Mensal']], on='ID Cliente')

faturamentoTotal = tabaux_faturamento_df['Valor Contrato Mensal'] * tabaux_faturamento_df['Tempo Total de Contrato (Meses)']
faturamentoTotal = faturamentoTotal.sum()

faturamentoTotal = 'R${:_.2f}'.format(faturamentoTotal)
faturamentoTotal = faturamentoTotal.replace('.', ',').replace('_', '.')
print("O faturamento total da empresa foi de: {}".format(faturamentoTotal))

O faturamento total da empresa foi de: R$5.519.160,00


### 3 - % Funcionários que Fecharam Contrato

In [47]:
percent_contratos_por_func = servicosPrestados_df['ID Funcionário'].unique()
#funcionarios_df.head()
percent_contratos_por_func = len(percent_contratos_por_func) / len(funcionarios_df)
print('{:.2%} dos funcionários fecharam contrato os clientes'.format(percent_contratos_por_func))

86.84% dos funcionários fecharam contrato os clientes


### 4 - Qtde de Contratos por Área

In [79]:
#Minha resolução para o problema quatro
qtdeContratosArea_df = servicosPrestados_df[['ID Funcionário','ID Cliente']].merge(funcionarios_df[['ID Funcionário','Area']])
qtdeContratosArea_df = qtdeContratosArea_df.drop_duplicates()
qtdeContratosArea_df = pd.DataFrame(data= qtdeContratosArea_df.groupby(['Area'])['ID Cliente'].count())
display(qtdeContratosArea_df)

,ID Cliente
Area,
Administrativo,63
Comercial,44
Financeiro,42
Logística,40
Operações,48


In [80]:
#Resolução do professor da Aula
contratos_area_df = servicosPrestados_df[['ID Funcionário']].merge(funcionarios_df[['ID Funcionário','Area']])
contratos_area_df = contratos_area_df['Area'].value_counts()
print(contratos_area_df)

Administrativo    63
Operações         48
Comercial         44
Financeiro        42
Logística         40
Name: Area, dtype: int64


### 5 - Total de Funcionários por área

In [83]:
funcionarioPorArea_df = funcionarios_df[['Area', 'ID Funcionário']]
funcionarioPorArea_df = funcionarioPorArea_df['Area'].value_counts()

print(funcionarioPorArea_df)

Administrativo    26
Comercial         26
Operações         23
Logística         21
Financeiro        18
Name: Area, dtype: int64


### 6 - Ticket médio mensal

In [93]:
clientes_df.head()
ticketMedioMes = clientes_df['Valor Contrato Mensal'].mean()

print('Ticket Médio mensal é de R${:,.2f}'.format(ticketMedioMes))

Ticket Médio mensal é de R$2,502.56
